# Libs and Inputs

In [20]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import time
from tqdm import tqdm
from GetIdxInOutRadious import GetIdxOutRadious, GetIdxInRadious
import matlab.engine
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm
from joblib import Parallel, delayed
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt


LCOE_Max=range(110,30,-5) #LCOEs investigated
Radious=20 #Radious for the energy collection system
NumWindTurbines=100 #Maximum number of wind turbines
NumKiteTurbines=100 #Maximum number of kite turbines

ShapeFileCoast="./GEO_data/ne_10m_coastline.shp"
ShapeFileStates="./GEO_data/ne_10m_admin_1_states_provinces_lines.shp"

WindKiteTrsData=np.load('PreprocessedData.npz',allow_pickle=True)

ResultsFileName='PortfolioOptimizationWindKite'


# Stage 1 Optimization

In [2]:
#LF kite and optimal kite design at each site location

## Prepare Data For the Optimization Model

In [21]:
#Wind Data
WindEnergy=WindKiteTrsData['WindEnergy']
WindLatLong=WindKiteTrsData['WindLatLong']
AnnualizedCostWind=WindKiteTrsData['AnnualizedCostWind']
MaxNumWindPerSite=WindKiteTrsData['MaxNumWindPerSite']

#Kite Data
KiteEnergy=WindKiteTrsData['KiteEnergy']
KiteLatLong=WindKiteTrsData['KiteLatLong']
AnnualizedCostKite=WindKiteTrsData['AnnualizedCostKite']
MaxNumKitesPerSite=WindKiteTrsData['MaxNumKitesPerSite']

#Transmission Data
Wind_AnnualizedCostTransmission=WindKiteTrsData['AnnualizedCostTransmission']
Wind_EfficiencyTransmission=WindKiteTrsData['EfficiencyTransmission']
Wind_MaxPowerTransmission=float(WindKiteTrsData['MaxPowerTransmission'])
Wind_TransLatLong=WindKiteTrsData['TransLatLong']

Kite_AnnualizedCostTransmission=WindKiteTrsData['AnnualizedCostTransmission']
Kite_EfficiencyTransmission=WindKiteTrsData['EfficiencyTransmission']
Kite_MaxPowerTransmission=float(WindKiteTrsData['MaxPowerTransmission'])
Kite_TransLatLong=WindKiteTrsData['TransLatLong']

TimeStepHours=WindKiteTrsData['TimeStepHours'] #Number of hours for each time step

In [22]:
#Vectorize maximum number of turbines per site location per technology
Nu=np.concatenate((MaxNumWindPerSite,MaxNumKitesPerSite))

#Vectorize annualized cost for each site location and per technology
AnnCost=np.concatenate((AnnualizedCostWind,AnnualizedCostKite)) #Annualized cost [$/Year]

#Vectorize energy generation in each site location and energy resource
EnergyGeneration=np.concatenate((WindEnergy,KiteEnergy),axis=0) #MW ()

NumWindSites=WindEnergy.shape[0]
NumKiteSites=KiteEnergy.shape[0]

NumTrasmissionSites=Wind_TransLatLong.shape[0]

#Compute variance covariance matrix
Sigma=np.cov(EnergyGeneration)#Variance covariance matrix [MWh**2]

#Step to guarantee that the matrix get SDP
for i in range(Sigma.shape[0]):
    Sigma[i,i]=Sigma[i,i]+10**-7

NumSites=Sigma.shape[0]

## Build Optimization Model Structure

In [23]:
MILP = ConcreteModel()

# Create Sets
MILP.SiteWind = RangeSet(0,NumWindSites-1)
MILP.SiteKite = RangeSet(0,NumKiteSites-1)
MILP.SiteTrs  = RangeSet(0,NumTrasmissionSites-1)
MILP.Y_Set    = RangeSet(0,NumWindSites+NumKiteSites-1)

# Create Variables
MILP.Y_Wind = Var(MILP.SiteWind, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
MILP.Y_Kite = Var(MILP.SiteKite, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
Y_Variable=[MILP.Y_Wind[i] for i in range(NumWindSites)]+[MILP.Y_Kite[i] for i in range(NumKiteSites)]

MILP.s_Wind = Var(MILP.SiteTrs, domain=Binary)# Binary variable to track the center of the energy collection system
MILP.s_Kite = Var(MILP.SiteTrs, domain=Binary)# Binary variable to track the center of the energy collection system

#Objective is to minimize the total variance in the energy generation
def objective_rule(MILP):   
    YtS=[(sum(MILP.Y_Wind[i]*Sigma[i,j] for i in MILP.SiteWind) + sum(MILP.Y_Kite[i]*Sigma[i+NumWindSites,j] for i in MILP.SiteKite))  for j in MILP.Y_Set]

    Total=sum(MILP.Y_Wind[i]*YtS[i] for i in MILP.SiteWind)+sum(MILP.Y_Kite[i]*YtS[i+NumWindSites] for i in MILP.SiteKite)

    return Total

MILP.OBJ = Objective(rule = objective_rule, sense=minimize)

#Constraints

#Maximum number of turbines per site location wind
def MaxTurbinesCell_Wind_rule(MILP,i):
    return MILP.Y_Wind[i]<=MaxNumWindPerSite[i]
MILP.Turbines_Cell_Wind = Constraint(MILP.SiteWind, rule=MaxTurbinesCell_Wind_rule)

#Maximum number of turbines per site location kite
def MaxTurbinesCell_Kite_rule(MILP,i):
    return MILP.Y_Kite[i]<=MaxNumKitesPerSite[i]
MILP.Turbines_Cell_Kite = Constraint(MILP.SiteKite, rule=MaxTurbinesCell_Kite_rule)

#---Choose center collection system - Start
MILP.ChooseOneCircle_Wind= Constraint(expr=sum(MILP.s_Wind[i] for i in MILP.SiteTrs)<=1)
MILP.ChooseOneCircle_Kite= Constraint(expr=sum(MILP.s_Kite[i] for i in MILP.SiteTrs)<=1)

#Get the sites that are in of the radious of the center of the collection system
IdxInWind=GetIdxInRadious(Wind_TransLatLong, WindLatLong, Radious)
IdxInKite=GetIdxInRadious(Kite_TransLatLong, KiteLatLong, Radious)


#---Choose center collection system - Start
####
def MaximumRadious_Wind(MILP,i):  
    SumWind_s=sum(MILP.Y_Wind[j] for j in IdxInWind[i])
    return SumWind_s>=MILP.s_Wind[i]*NumWindTurbines

MILP.Maximum_Radious_Wind = Constraint(MILP.SiteTrs, rule=MaximumRadious_Wind)

####
def MaximumRadious_Kite(MILP,i):  
    SumKite_s=sum(MILP.Y_Kite[j] for j in IdxInKite[i])
    return SumKite_s>=MILP.s_Kite[i]*NumKiteTurbines

MILP.Maximum_Radious_Kite = Constraint(MILP.SiteTrs, rule=MaximumRadious_Kite)
#---Choose center collection system - End

MILP.SetNumWindTurbines= Constraint(expr=sum(MILP.Y_Wind[i] for i in MILP.SiteWind)==NumWindTurbines)
MILP.SetNumKiteTurbines= Constraint(expr=sum(MILP.Y_Kite[i] for i in MILP.SiteKite)==NumKiteTurbines)


#LCOE Target
def LCOETarget_S1(MILP,LCOE_Max):  
    EG_Wind=sum(MILP.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP.SiteWind) #Average MW for wind
    EG_Kite=sum(MILP.Y_Kite[i]*np.average(KiteEnergy[i,:]) for i in MILP.SiteKite) #Average MW for kite
    
    MWh=(EG_Kite+EG_Wind)*24*365 #MWh

    Cost_Wind=sum(MILP.Y_Wind[i]*AnnualizedCostWind[i] for i in MILP.SiteWind)+sum(MILP.s_Wind[i]*Wind_AnnualizedCostTransmission[i] for i in MILP.SiteTrs)
    Cost_Kite=sum(MILP.Y_Kite[i]*AnnualizedCostKite[i] for i in MILP.SiteKite)+sum(MILP.s_Kite[i]*Kite_AnnualizedCostTransmission[i] for i in MILP.SiteTrs)

    Cost=Cost_Wind+Cost_Kite

    return Cost<=LCOE_Max*MWh  
#
opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.05
opt.options['Method'] = 5
#opt.options['max_iter'] = 500

In [27]:
results=opt.solve(MILP, tee=True)

Set parameter MIPGap to value 0.05
Set parameter Method to value 5
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3546 rows, 3541 columns and 27507 nonzeros
Model fingerprint: 0xd324ed68
Model has 2941533 quadratic objective terms
Variable types: 0 continuous, 3541 integer (1108 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+07]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 3323 rows and 890 columns
Presolve time: 0.22s
Presolved: 223 rows, 2651 columns, 21839 nonzeros
Presolved model has 2941533 quadratic objective terms
         Using dual simplex
Variable types: 0 continuous, 2651 integer (218 binary)

Root relaxation: infeasible, 2740 iterations, 0.35 seconds (1.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl

## Solve Portfolio Optimization Stage 1 

In [22]:
SaveFeasibility, Save_LCOETarget, Save_LCOE_Achieved, SaveVariance = list(), list(), list(), list()
SaveYWind, SaveYKite, SaveYTrans_Wind, SaveYTrans_Kite = list(), list(), list(), list()
Save_LCOE_Kite, Save_LCOE_Wind= list(), list()

LowestLCOE=10**10
for LCOE_Idx in tqdm(range(len(LCOE_Max))):
    LCOETarget=LCOE_Max[LCOE_Idx]
    
    if LCOETarget<LowestLCOE:    
        Bypass=0
        #Upperbound For the LCOE Activate Constraint
        LCOE_Target=LCOETarget_S1(MILP,LCOETarget)
        MILP.LCOE_Target = Constraint(rule=LCOE_Target)
        print("Running Model With LCOE= %.2f" % LCOETarget)
        
        try:
            results=opt.solve(MILP, tee=False)
        except:
            Bypass=1
            MILP.del_component(MILP.LCOE_Target)  
    
        if Bypass==0:
            if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
                SaveFeasibility.append(1)
                Save_LCOETarget.append(LCOETarget)
                
                Optimal_Y_Kite =np.array([MILP.Y_Kite.get_values()[j] for j in MILP.SiteKite])
                Optimal_Y_Wind =np.array([MILP.Y_Wind.get_values()[j] for j in MILP.SiteWind])
                Optimal_Y_Trans_Wind=np.array([MILP.s_Wind.get_values()[j] for j in MILP.SiteTrs])
                Optimal_Y_Trans_Kite=np.array([MILP.s_Kite.get_values()[j] for j in MILP.SiteTrs])


                SaveYWind.append(Optimal_Y_Wind)
                SaveYKite.append(Optimal_Y_Kite)
                SaveYTrans_Wind.append(Optimal_Y_Trans_Wind)
                SaveYTrans_Kite.append(Optimal_Y_Trans_Kite)

                #Current LCOE
                EG_Wind=np.sum(Optimal_Y_Wind*np.average(WindEnergy,axis=1)) #MW avg
                EG_Kite=np.sum(Optimal_Y_Kite*np.average(KiteEnergy,axis=1)) #MW avg

                MWh=(EG_Kite+EG_Wind)*24*365 #MWh

                Cost_Wind  = sum(Optimal_Y_Wind*AnnualizedCostWind)+sum(Optimal_Y_Trans_Wind*Wind_AnnualizedCostTransmission)
                Cost_Kite  = sum(Optimal_Y_Kite*AnnualizedCostKite)+ sum(Optimal_Y_Trans_Kite*Kite_AnnualizedCostTransmission)


                Cost=Cost_Wind+Cost_Kite
                CurrentLCOE=Cost/MWh
                
                Save_LCOE_Achieved.append(CurrentLCOE)
                SaveTotalMWAvg.append(value(MILP.OBJ))

                LCOE_Wind=Cost_Wind/(EG_Wind*24*365)
                LCOE_Kite=Cost_Kite/(EG_Kite*24*365)

                Save_LCOE_Wind.append(LCOE_Wind)
                Save_LCOE_Kite.append(LCOE_Kite)


                LowestLCOE=CurrentLCOE
                
                print("LCOE Wind: %.2f,\nLCOE Kite: %.2f\n" % (LCOE_Wind,LCOE_Kite))

                
                #Delete constraint for its modification in the next step of the for loop
                MILP.del_component(MILP.LCOE_Target)
            
            else:# Something else is wrong
                MILP.del_component(MILP.LCOE_Target)
                SaveFeasibility.append(0)
                Save_LCOETarget.append(None)
                Save_LCOE_Achieved.append(None)
                SaveYWind.append(None)
                SaveYKite.append(None)
                SaveYTrans_Kite.append(None)
                SaveYTrans_Wind.append(None)
                Save_LCOE_Wind.append(None)
                Save_LCOE_Kite.append(None)    
   

                break


  0%|          | 0/40 [00:00<?, ?it/s]

Running Model With LCOE= 110.00


  2%|▎         | 1/40 [01:32<1:00:26, 92.98s/it]

Running Model With LCOE= 108.00


In [126]:
#Save Results
np.savez("./ResultsPortOpt/"+ResultsFileName +"Stage1LF"+ ".npz", 
        #Wind Data
        WindEnergy=WindEnergy,
        WindLatLong=WindLatLong,
        AnnualizedCostWind=AnnualizedCostWind,
        MaxNumWindPerSite=MaxNumWindPerSite,
        #Kite Data
        KiteEnergy=KiteEnergy,
        KiteLatLong=KiteLatLong,
        AnnualizedCostKite=AnnualizedCostKite,
        MaxNumKitesPerSite=MaxNumKitesPerSite,
        #Transmission Data
        AnnualizedCostTransmission=AnnualizedCostTransmission,
        TransLatLong=TransLatLong,
        EfficiencyTransmission=EfficiencyTransmission,
        MaxPowerTransmission=MaxPowerTransmission,
        TimeStepHours=TimeStepHours,
        #Solutions
        SaveFeasibility=SaveFeasibility,
        Save_LCOETarget=Save_LCOETarget,
        Save_LCOE_Achieved=Save_LCOE_Achieved,
        SaveYWind=SaveYWind,
        SaveYKite=SaveYKite,
        SaveYTrans_Kite=SaveYTrans_Kite,
        SaveYTrans_Wind=SaveYTrans_Wind,
        Save_LCOE_Wind=Save_LCOE_Wind,
        Save_LCOE_Kite=Save_LCOE_Kite)


c:\Users\Remote\anaconda3\envs\Gurobi\lib\site-packages\numpy\lib\npyio.py:696: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


# Stage 2 Optimization

## Convert Stage 1 Solution to Stage 2 Input

In [127]:
#Data for Stage 2
CaseInvestigated=2
MaxNumDesigns=1
CaseKite_Idx=np.where(SaveYKite[CaseInvestigated])[0]
CaseTrans_Idx=np.where(SaveYTrans[CaseInvestigated])[0][0]

IdxInWind=GetIdxInRadious(TransLatLong[CaseTrans_Idx,:], WindLatLong, Radious) #Kites inside radious of energy collection system
IdxInKite=GetIdxInRadious(TransLatLong[CaseTrans_Idx,:], KiteLatLong, Radious) #Kites inside radious of energy collection system

#transmission location
TransLoc=np.where(SaveYTrans[CaseInvestigated])[0][0]

#update wind data
NumWindSites_s2=len(IdxInWind)
WindEnergy_s2=WindEnergy[IdxInWind,:]
WindLatLong_s2=WindLatLong[IdxInWind,:]
AnnualizedCostWind_s2=AnnualizedCostWind[IdxInWind]
MaxNumWindPerSite_s2=MaxNumWindPerSite[IdxInWind]

#update kite data
NumKiteSites_s2=len(IdxInKite)
NumKiteDesigns_s2=len(IdxInKite)
MaxNumKitesPerSite_s2=MaxNumKitesPerSite[IdxInKite]
KiteLatLong_s2=KiteLatLong[IdxInKite,:]
AnnualizedCostKite_s2=AnnualizedCostKite[IdxInKite]

KiteEnergy_s2=np.zeros((NumKiteSites_s2,NumKiteDesigns_s2,NumTimeSteps))

DataStage2_Kite=[]
for i in range(len(IdxInKite)):
    site=IdxInKite[i]
    for j in range(len(IdxInKite)):
        design=IdxInKite[j]

        DataStage2_Kite.append({"X":WindKiteTrsData["X_Y_Vecs"][site,0],
                                "Y":WindKiteTrsData["X_Y_Vecs"][site,1],
                                "Desgin":design,
                                "Site":site,
                                "uopt": WindKiteTrsData["uopt_vecs"][design,:],
                                "SiteIdx_s2":i,
                                "DesignIdx_s2":j})   


In [ ]:
# I have 64GB of RAM, so I can run 10 envs at the same time. 
#2500 runs = 30 minutes

def UpdateGenerationTS (Data, NumEnvs=15):
    Envs=[matlab.engine.start_matlab() for i in range(NumEnvs)] #Create envs
    for i in range(NumEnvs):
        Envs[i].cd(r'C:\Users\Remote\Desktop\Projects\OceanProject4_1_Ver\PortOpt_KiteWind_MaxGenLCOE\KiteLF_Optimization', nargout=0)
        Envs[i].eval("File='DataSetPlatform.mat';"+"load(File);" ,nargout=0)

    cmd1=[]
    cmd2=[]

    for i in range(len(Data)):
        X=Data[i]["X"]
        Y=Data[i]["Y"]
        uopt  =Data[i]["uopt"]

        cmd1_tmp = str('xSite =['+str(X)+','+str(Y)+'];')
        cmd2_tmp = str('uGeo =['+str(uopt[0])+','+str(uopt[1])+','+str(uopt[2])+','+str(uopt[3])+','+str(15000)+'];')
        cmd1.append(cmd1_tmp)
        cmd2.append(cmd2_tmp)

    G_count=0
    NumRuns=0
    while G_count!=len(Data):

        G_count=G_count+NumRuns

        print("%.2f %% complete" % (G_count/len(Data)*100))

        NumRuns=np.min([len(Envs),len(Data)-G_count]) #Number of runs to do in parallel
        
        for i in range(0,NumRuns,1):
            Envs[i].eval(cmd1[G_count+i],nargout=0) #Inputs
            Envs[i].eval(cmd2[G_count+i],nargout=0) #Inputs

        #Separate loop to run in parallel (only for running in parallel)
        for i in range(0,NumRuns,1):
            Envs[i].powerFunc_Python(nargout=0,background=True) #Solve problem in background

        for i in range(0,NumRuns,1):   
            Jopt_vec  =Envs[i].workspace["Jopt_vec"][0]
            theta_vec =Envs[i].workspace["theta_vec"][0]
            l_vec     =Envs[i].workspace["l_vec"][0]

            Data[G_count+i]["Jopt_vec"]=Jopt_vec
            Data[G_count+i]["theta_vec"]=theta_vec
            Data[G_count+i]["l_vec"]=l_vec

    for i in range(NumEnvs):
        Envs[i].quit()

    return Data

Data=UpdateGenerationTS(DataStage2_Kite)

In [ ]:
for i in range(len(Data)):
    SiteIdx_s2=Data[i]["SiteIdx_s2"]
    DesignIdx_s2=Data[i]["DesignIdx_s2"]
    KiteEnergy_s2[SiteIdx_s2,DesignIdx_s2,:]=np.asarray(Data[i]["Jopt_vec"])[0]/1000 #MW

## Optimize with limited designs

In [ ]:
MILP_s2 = ConcreteModel()
BM=BigM
# Create Sets
MILP_s2.SiteWind = RangeSet(0,NumWindSites_s2-1)
MILP_s2.SiteKite = RangeSet(0,NumKiteSites_s2-1)
MILP_s2.Designs  = RangeSet(0,NumKiteDesigns_s2-1)
MILP_s2.TimeSteps = RangeSet(0,NumTimeSteps-1)

# Create Variables
MILP_s2.Y_Wind = Var(MILP_s2.SiteWind, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location
MILP_s2.Y_Kite = Var(MILP_s2.SiteKite, MILP_s2.Designs, domain=NonNegativeIntegers)# Integer variable to track the number of wind turbines used per site location

MILP_s2.w = Var(MILP_s2.Designs , domain=Binary)# Binary variable to track the center of the energy collection system

MILP_s2.Delta = Var(MILP_s2.TimeSteps, domain=NonNegativeReals) #Curtailment variable

#Objective Function
def objective_rule(MILP_s2):   
    EGWind=sum(MILP_s2.Y_Wind[i]*np.average(WindEnergy_s2[i,:]) for i in MILP_s2.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP_s2.Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Energy generation from kite turbines

    TotalCurtailment=sum(MILP_s2.Delta[t] for t in MILP_s2.TimeSteps)/NumTimeSteps #Average curtailment MW

    Obj=(EGWind + EGKite - TotalCurtailment)#*24*365#Mwh/year
    return Obj

MILP_s2.OBJ = Objective(rule = objective_rule, sense=maximize)

#Constraints
#Maximum number of turbines per site location wind
def MaxTurbinesCell_Wind_rule(MILP_s2,i):
    return MILP_s2.Y_Wind[i]<=MaxNumWindPerSite_s2[i]
MILP_s2.Turbines_Cell_Wind = Constraint(MILP_s2.SiteWind, rule=MaxTurbinesCell_Wind_rule)

#Maximum number of turbines per site location kite
def MaxTurbinesCell_Kite_rule(MILP_s2,i):
    return sum(MILP_s2.Y_Kite[i,d] for d in MILP_s2.Designs) <= MaxNumKitesPerSite_s2[i]
MILP_s2.Turbines_Cell_Kite = Constraint(MILP_s2.SiteKite, rule=MaxTurbinesCell_Kite_rule)

#Curtailment constraint
def Curtailment_rule(MILP_s2,t):
    EGWind=sum(MILP_s2.Y_Wind[i]*WindEnergy_s2[i,t] for i in MILP_s2.SiteWind) #Energy generation from wind turbines
    EGKite=sum(MILP_s2.Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Energy generation from kite turbines
    return -MILP_s2.Delta[t]+ EGWind+ EGKite<=MaxPowerTransmission
MILP_s2.Curtailment = Constraint(MILP_s2.TimeSteps, rule=Curtailment_rule)

#---Choose center collection system - End

#LCOE Target
def LCOETarget_S1(MILP_s2,LCOE_Max):  
    EG_Wind=sum(MILP_s2.Y_Wind[i]*np.average(WindEnergy[i,:]) for i in MILP_s2.SiteWind) #Average MW for wind
    EG_Kite=sum(MILP_s2.Y_Kite[i,d]*np.average(KiteEnergy_s2[i,d,:]) for i in MILP_s2.SiteKite for d in MILP_s2.Designs) #Average MW for kite
    TotalCurtailment=sum(MILP_s2.Delta[i] for i in MILP_s2.TimeSteps)/NumTimeSteps #Average curtailment MW
    
    MWh=(EG_Kite+EG_Wind-TotalCurtailment)*24*365 #MWh

    Cost_Wind=sum(MILP_s2.Y_Wind[i]*AnnualizedCostWind[i] for i in MILP_s2.SiteWind)
    Cost_Kite=sum(MILP_s2.Y_Kite[i,d]*AnnualizedCostKite[d] for i in MILP_s2.SiteKite for d in MILP_s2.Designs)
    Cost_Transmission=AnnualizedCostTransmission[TransLoc]
    Cost=Cost_Wind+Cost_Kite+Cost_Transmission

    return Cost<=LCOE_Max*MWh # 300 is a big M for the total number of turbines installed       

LCOE_Target=LCOETarget_S1(MILP_s2,LCOETarget)
MILP_s2.LCOE_Target = Constraint(rule=LCOE_Target)

def TrackDesigns(MILP_s2,d):  
    return sum(MILP_s2.Y_Kite[i,d] for i in MILP_s2.SiteKite)<=MILP_s2.w[d]*BM 
MILP_s2.TrackDesigns = Constraint(MILP_s2.Designs, rule=TrackDesigns)

def CountDesigns(MILP_s2):  
    return sum(MILP_s2.w[d] for d in MILP_s2.Designs)<=MaxNumDesigns
MILP_s2.CountDesigns = Constraint(rule=CountDesigns)
#---Choose center collection system - End
#
opt = SolverFactory('gurobi', solver_io="python")
opt.options['mipgap'] = 0.05
#opt.options['max_iter'] = 500
results=opt.solve(MILP, tee=True)

Set parameter MIPGap to value 0.05
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 3075 rows, 3074 columns and 1546196 nonzeros
Model fingerprint: 0xeb034278
Variable types: 87 continuous, 2987 integer (554 binary)
Coefficient statistics:
  Matrix range     [3e-10, 1e+04]
  Objective range  [1e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 19.9766393
Presolve removed 2946 rows and 2658 columns
Presolve time: 0.38s
Presolved: 129 rows, 416 columns, 40153 nonzeros
Variable types: 87 continuous, 329 integer (19 binary)
Found heuristic solution: objective 21.8061166

Root relaxation: objective 1.000000e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds 